# iterate over folder with images, count images with the same size

In [19]:
import cv2, os

img= cv2.imread('data/DIV2K_train_HR/0001.png')
img.shape

(1404, 2040, 3)

In [29]:
from tqdm.notebook import tqdm


path = "data/DIV2K_train_HR"
sizes={}
for file in tqdm(os.listdir("data/DIV2K_train_HR")):
    image = cv2.imread(path + '/' + file)
    if image.shape in sizes.keys():
        sizes[image.shape]+=1
    else:
        sizes[image.shape]=1
    


  0%|          | 0/800 [00:00<?, ?it/s]

In [31]:
dict(sorted(sizes.items(), key=lambda item: item[0]))

{(648, 2040, 3): 1,
 (732, 2040, 3): 1,
 (744, 2040, 3): 1,
 (768, 2040, 3): 1,
 (780, 2040, 3): 1,
 (852, 2040, 3): 1,
 (864, 2040, 3): 1,
 (888, 2040, 3): 2,
 (936, 2040, 3): 1,
 (948, 2040, 3): 1,
 (960, 2040, 3): 3,
 (972, 2040, 3): 1,
 (1044, 2040, 3): 1,
 (1068, 2040, 3): 2,
 (1080, 2040, 3): 1,
 (1092, 2040, 3): 1,
 (1128, 2040, 3): 1,
 (1140, 2040, 3): 9,
 (1152, 2040, 3): 20,
 (1164, 2040, 3): 3,
 (1176, 2040, 3): 2,
 (1188, 2040, 3): 7,
 (1200, 2040, 3): 2,
 (1212, 2040, 3): 4,
 (1224, 2040, 3): 11,
 (1248, 2040, 3): 3,
 (1260, 2040, 3): 8,
 (1272, 2040, 3): 6,
 (1284, 2040, 3): 3,
 (1296, 2040, 3): 6,
 (1308, 2040, 3): 4,
 (1320, 2040, 3): 5,
 (1332, 2040, 3): 10,
 (1344, 2040, 3): 9,
 (1356, 2040, 3): 429,
 (1368, 2040, 3): 28,
 (1380, 2040, 3): 3,
 (1392, 2040, 3): 5,
 (1404, 2040, 3): 1,
 (1416, 2040, 3): 1,
 (1428, 2040, 3): 4,
 (1440, 2040, 3): 3,
 (1452, 2040, 3): 8,
 (1464, 2040, 3): 6,
 (1476, 2040, 3): 1,
 (1488, 2040, 3): 3,
 (1512, 2040, 3): 2,
 (1524, 2040, 3): 3

## Results
 There is 429 images of size (1356, 2040, 3)

In [ ]:
# pip install fiftyone
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["segmentations"],
    classes=["cat", "dog"],
    max_samples=25,
)

session = fo.launch_app(dataset)

In [14]:
import math
from torch import nn

class ESPCN(nn.Module):
    def __init__(self, scale_factor, num_channels=1):
        super(ESPCN, self).__init__()
        self.first_part = nn.Sequential(
            nn.Conv2d(num_channels, 64, kernel_size=5, padding=5//2),
            nn.Tanh(),
            nn.Conv2d(64, 32, kernel_size=3, padding=3//2),
            nn.Tanh(),
        )
        self.last_part = nn.Sequential(
            nn.Conv2d(32, num_channels * (scale_factor ** 2), kernel_size=3, padding=3 // 2),
            nn.PixelShuffle(scale_factor)
        )

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m.in_channels == 32:
                    nn.init.normal_(m.weight.data, mean=0.0, std=0.001)
                    nn.init.zeros_(m.bias.data)
                else:
                    nn.init.normal_(m.weight.data, mean=0.0, std=math.sqrt(2/(m.out_channels*m.weight.data[0][0].numel())))
                    nn.init.zeros_(m.bias.data)

    def forward(self, x):
        x = self.first_part(x)
        x = self.last_part(x)
        return x